In [1]:
import gradio as gr
import joblib
import os
import numpy as np
import pandas as pd
import gdown
import requests
from bs4 import BeautifulSoup
import re
import time

# --- 1. 모델 가중치 다운로드 및 로드 ---

# 모델 가중치 다운로드 함수
def download_model():
    # 파일 ID는 예시 코드에 있던 것을 사용합니다.
    file_id = '138_2NXbeGm2wCCaOVeDd12c7hGrm7alE'
    destination = 'news_classifier_model.pkl'

    if not os.path.exists(destination):
        print(f"📥 모델 다운로드 중... ({destination})")
        url = f'https://drive.google.com/uc?id={file_id}'
        gdown.download(url, destination, quiet=False)
    return destination

# 모델 로드 (앱 실행 시 1회 수행)
try:
    model_path = download_model()
    saved_object = joblib.load(model_path)
    model = saved_object['model']
    le = saved_object['encoder']
    print("✅ 모델 로드 완료")
except Exception as e:
    print(f"❌ 모델 로드 중 오류 발생: {e}. 예측 기능이 작동하지 않습니다.")
    model, le = None, None

# --- 2. 뉴스 수집 및 재분류 설정 ---

CS_KEYWORDS = [
    'ai', 'artificial intelligence', 'tech', 'technology', 'digital',
    'data', 'cyber', 'crypto', 'semiconductor', 'nvidia', 'chip',
    'google', 'microsoft', 'apple', 'meta', 'algorithm', 'software',
    'cloud', 'robot', 'ev', 'electric vehicle'
]

def get_news_data():
    """RSS 피드에서 뉴스 헤드라인, 원본 카테고리, 그리고 ⭐링크⭐를 수집하여 데이터프레임으로 반환"""
    rss_urls = {
        'Politics_NYT': 'https://rss.nytimes.com/services/xml/rss/nyt/Politics.xml',
        'Economy_BBC': 'http://feeds.bbci.co.uk/news/business/rss.xml',
        'Technology_NYT': 'https://rss.nytimes.com/services/xml/rss/nyt/Technology.xml',
        'Technology_Wired': 'https://www.wired.com/feed/rss',
        'Society_BBC': 'http://feeds.bbci.co.uk/news/health/rss.xml',
        'Sports_BBC': 'http://feeds.bbci.co.uk/news/sport/rss.xml'
    }

    news_data = []
    print("\n🚀 뉴스 데이터 수집을 시작합니다...")

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124 Safari/537.36"
    }

    for key, url in rss_urls.items():
        category_name = key.split('_')[0] # 원본 카테고리 추출

        try:
            response = requests.get(url, headers=headers, timeout=5)
            # RSS 파싱
            soup = BeautifulSoup(response.content, 'xml')
            items = soup.find_all('item')
            print(f"[{category_name}] {len(items)}개 발견 ({key})")

            for item in items:
                title_tag = item.find('title')
                link_tag = item.find('link') # ⭐ link 태그 찾기 ⭐

                title = title_tag.text.strip() if title_tag else None
                link = link_tag.text.strip() if link_tag else None

                if title and link: # 제목과 링크가 모두 있어야 유효한 기사로 간주
                    # 'headline', 'original_category', 'link'를 모두 저장
                    news_data.append({'headline': title, 'original_category': category_name, 'link': link})

        except Exception as e:
            print(f"Error fetching {key}: {e}")

    if not news_data:
        raise Exception("❌ 수집된 데이터가 없습니다.")

    df = pd.DataFrame(news_data)
    # 중복 헤드라인 제거 시 원본 카테고리는 유지됨
    df = df.drop_duplicates(subset=['headline']).reset_index(drop=True)
    print(f"\n✅ 총 {len(df)}개 고유 데이터 수집 완료!")
    return df

# --- 3. 분류 함수 재정의 ---

# (기존) 단일 텍스트 예측 함수
def predict_text(headline):
    if model is None:
        return {"Error": "모델이 로드되지 않았습니다."}

    pred_proba = model.predict_proba([headline])[0]
    class_names = le.classes_
    confidences = {class_names[i]: float(pred_proba[i]) for i in range(len(class_names))}
    return confidences

# (통합) 데이터 수집 및 대량 분석 함수
def collect_and_analyze():
    """
    1. 뉴스 데이터를 수집합니다.
    2. 수집된 데이터에 대해 모델 예측을 수행합니다.
    3. 요약 메시지, CS_Insight 원본 카테고리 분석, 통계, 필터링된 데이터를 반환합니다.
    """
    # 에러 처리 시 반환할 빈/에러 DataFrame
    empty_df = pd.DataFrame()
    if model is None:
        error_msg = "❌ 모델 로드 오류: 예측 기능을 사용할 수 없습니다."
        # 5개의 출력을 모두 반환
        return error_msg, empty_df, empty_df, empty_df, empty_df

    try:
        # 1. 데이터 수집 ('headline', 'original_category', 'link' 포함)
        df = get_news_data()

        # 2. 예측 수행
        headlines = df['headline'].astype(str).tolist()
        predictions = model.predict(headlines)
        pred_labels = le.inverse_transform(predictions)

        # 3. 결과 데이터프레임 생성 (link 포함)
        result_df = df.copy()
        result_df['Predicted_Category'] = pred_labels

        # 4. 통계 (Predicted_Category별 개수)
        stats = result_df['Predicted_Category'].value_counts().reset_index()
        stats.columns = ['Category', 'Count']

        # 5. CS_Insight 필터링 (link 포함)
        cs_news = result_df[result_df['Predicted_Category'] == 'CS_Insight']

        # 6. CS_Insight 원본 카테고리 분석
        original_stats = cs_news['original_category'].value_counts().reset_index()
        original_stats.columns = ['Original_Category', 'CS_Count']

        # 7. 요약 메시지 생성
        total_count = len(result_df)
        cs_count = len(cs_news)
        summary_message = f"**분석 요약:** 총 **{total_count}**개 뉴스 수집 및 분류 완료. 그 중 **CS_Insight 뉴스는 {cs_count}개** 입니다."

        print("\n📊 분석 완료!")
        print(stats)

        # 요약 메시지(string), CS_Insight 원본 카테고리 통계 DF, Predicted_Category 통계 DF, CS_Insight 뉴스 DF, 전체 결과 DF 순으로 반환 (총 5개)
        return summary_message, original_stats, stats, cs_news, result_df

    except Exception as e:
        error_msg = f"에러 발생: {e}"
        print(error_msg)
        error_df = pd.DataFrame({'Error': [error_msg]})
        # 에러 발생 시 5개의 출력을 모두 반환 (메시지 + 4개의 에러 DF)
        return error_msg, error_df, error_df, error_df, error_df

# --- 4. Gradio 인터페이스 구성 (탭 순서 변경) ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 📰 CS_Insight News Classifier & Analyzer")
    gr.Markdown("CS 전공자를 위한 뉴스 분류기입니다. **실시간 뉴스 수집 & 분류** 기능을 사용해 보세요.")

    # ⭐ 1. 실시간 데이터 수집 및 분석 탭 (메인으로 배치)
    with gr.Tab("📡 실시간 뉴스 수집 및 분류"):
        gr.Markdown("버튼을 클릭하면 설정된 RSS 피드에서 최신 뉴스를 수집하고, 모델로 분류합니다.")
        gr.Markdown("🔗 **참고:** 아래 'CS_Insight 뉴스 모아보기'와 '전체 분석 결과' 테이블에는 **Link** 컬럼이 포함되어 있습니다. URL을 복사하여 원본 기사를 확인하세요.") # ⭐ 링크 안내 추가

        btn_collect_analyze = gr.Button("뉴스 수집 및 분류 시작", variant="primary")

        summary_output = gr.Markdown("분석을 시작해주세요.")

        original_stats_output = gr.Dataframe(label="⭐ CS_Insight 원본 카테고리 분석",
                                             type="pandas",
                                             headers=["Original Category", "CS Count"])

        with gr.Row():
            stats_output = gr.Dataframe(label="📊 전체 Predicted 카테고리별 통계", type="pandas", headers=["Category", "Count"])

        with gr.Row():
            # ⭐ Link 컬럼이 포함된 DataFrame 출력
            cs_output = gr.Dataframe(label="💎 CS_Insight 뉴스 모아보기", type="pandas", wrap=True)

        with gr.Accordion("전체 분석 결과 보기 (클릭해서 펼치기)", open=False):
            # ⭐ Link 컬럼이 포함된 DataFrame 출력
            full_output = gr.Dataframe(label="전체 데이터", type="pandas", wrap=True)

        btn_collect_analyze.click(
            collect_and_analyze,
            inputs=[],
            outputs=[summary_output, original_stats_output, stats_output, cs_output, full_output]
        )

    # ⭐ 2. 텍스트 입력 분석 탭 (두 번째로 배치)
    with gr.Tab("📝 텍스트 입력 분석"):
        with gr.Row():
            txt_input = gr.Textbox(lines=2, placeholder="뉴스 헤드라인을 입력하세요...")
            txt_output = gr.Label(num_top_classes=3, label="분류 결과")
        btn_predict = gr.Button("분류하기")

        examples = [
            ["NVIDIA announces new Blackwell AI chip for data centers"],
            ["Federal Reserve raises interest rates to fight inflation"],
            ["Premier League: Liverpool beats Chelsea in a thrilling match"],
            ["Tech stocks tumble as Apple and Google face regulatory pressure"]
        ]
        gr.Examples(examples, inputs=txt_input)

        btn_predict.click(predict_text, inputs=txt_input, outputs=txt_output)

# --- 5. 앱 실행 ---
if __name__ == "__main__":
    demo.launch(share=True)

📥 모델 다운로드 중... (news_classifier_model.pkl)


Downloading...
From: https://drive.google.com/uc?id=138_2NXbeGm2wCCaOVeDd12c7hGrm7alE
To: /content/news_classifier_model.pkl
100%|██████████| 4.08M/4.08M [00:00<00:00, 32.5MB/s]


✅ 모델 로드 완료


/tmp/ipython-input-1817574945.py:167: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://984444f7739d56ecb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
